In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyproj
!pip install haversine
!pip install gurobipy
!pip install shapely
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 32.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 56.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 51.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import pyproj

In [4]:
# 서울시 전동킥보드 주차구역 현황
es_park = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울시 전동킥보드 주차구역 현황.csv', encoding = 'euc-kr')
es_park = es_park.drop(['거치대 크기'], axis=1)

In [5]:
# 전동킥보드 거치대 Y 주차 현황
es_park_y = es_park[es_park['거치대 유무'].str.contains('Y')]
# 전동킥보드 거치대 N 주차 현황
es_park_n = es_park[es_park['거치대 유무'].str.contains('N')]

In [6]:
es_park_y.head()

,순번,시군구명,주소,상세위치,거치대 유무
0,1,종로구,팔판동 115-63,청와대 춘추문 맞은편 인근,Y
1,2,종로구,연건동 218-1,KT광화문 혜화지사 앞,Y
2,3,종로구,연건동 178-3,홍익대학교 대학로 맞은편,Y
3,4,종로구,동승동 1-24,대학로 마로니에공원 앞,Y
4,5,종로구,와룡동 75-4,연악사 맞은편,Y


In [7]:
es_park_n.head()

,순번,시군구명,주소,상세위치,거치대 유무
21,22,용산구,용산구 갈월동 69-29,숙대입구역 10번 출구 앞,N
26,27,성동구,홍익동 614-1,상왕십리역 1번 출구 따릉이 거치대 인근,N
27,28,광진구,자양동854,뚝섬유원지역 1번 축구 인근,N
28,29,동대문구,장안동 416-10,창도빌딩 앞(지하철 환풍구 옆 가로등 가로수 사이),N
29,30,동대문구,답십리동 497-23,천호대로 257 청계푸르지오시티오피스텔 앞,N


In [8]:
es_address= es_park_n['주소']
es_address.head()

21    용산구 갈월동 69-29
26        홍익동 614-1
27           자양동854
28       장안동 416-10
29      답십리동 497-23
Name: 주소, dtype: object

In [9]:
####### 도로명주소 위도 경도 값으로 바꿔주기 ########
from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(es_address):
    try:
        geo = geo_local.geocode(es_address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

In [10]:
#####주소를 위,경도 값으로 변환하기 #####
##### 오래걸림 ######
latitude = []
longitude =[]

for i in es_address:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

In [11]:
#####Dataframe만들기######
es_address_df = pd.DataFrame({'시군구명': es_park_n['시군구명'],'주소':es_park_n['주소'], '상세위치':es_park_n['상세위치'], '위도':latitude, '경도':longitude})

In [12]:
es_address_df

,시군구명,주소,상세위치,위도,경도
21,용산구,용산구 갈월동 69-29,숙대입구역 10번 출구 앞,37.544870,126.971544
26,성동구,홍익동 614-1,상왕십리역 1번 출구 따릉이 거치대 인근,37.566014,127.032728
27,광진구,자양동854,뚝섬유원지역 1번 축구 인근,0.000000,0.000000
28,동대문구,장안동 416-10,창도빌딩 앞(지하철 환풍구 옆 가로등 가로수 사이),37.571050,127.068060
29,동대문구,답십리동 497-23,천호대로 257 청계푸르지오시티오피스텔 앞,37.570020,127.054670
...,...,...,...,...,...
183,송파구,송파동 1-1,석촌역 4번 출구 뒤,37.504790,127.113690
184,송파구,가락동 160-5,오금역 4번 출구 앞,37.496720,127.117440
185,송파구,잠실동 270,"잠실새내역 3, 4번 출구 사이",37.512281,127.085599
188,강동구,명일동 46-5,고덕역 4번 출구 인근,37.548410,127.151580


In [13]:
# csv파일 저장
es_address_df.to_csv("es_address_df.csv", index = False)

In [14]:
es_address_df = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/df.to_csv/es_address_df.csv')

In [15]:
es_address_df.isnull().sum()

시군구명    0
주소      0
상세위치    0
위도      0
경도      0
dtype: int64

In [16]:
# 위도가 0인 열 제거 (전처리)
es_address_df = es_address_df[es_address_df['위도'] != 0]
es_address_df

,시군구명,주소,상세위치,위도,경도
0,용산구,용산구 갈월동 69-29,숙대입구역 10번 출구 앞,37.544870,126.971544
1,성동구,홍익동 614-1,상왕십리역 1번 출구 따릉이 거치대 인근,37.566014,127.032728
3,동대문구,장안동 416-10,창도빌딩 앞(지하철 환풍구 옆 가로등 가로수 사이),37.571050,127.068060
4,동대문구,답십리동 497-23,천호대로 257 청계푸르지오시티오피스텔 앞,37.570020,127.054670
5,동대문구,신설동 98-32,신설동역 5번출구 뒤 자전거보관대 옆(파리바게트 앞),37.573478,127.025707
...,...,...,...,...,...
128,송파구,송파동 1-1,석촌역 4번 출구 뒤,37.504790,127.113690
129,송파구,가락동 160-5,오금역 4번 출구 앞,37.496720,127.117440
130,송파구,잠실동 270,"잠실새내역 3, 4번 출구 사이",37.512281,127.085599
131,강동구,명일동 46-5,고덕역 4번 출구 인근,37.548410,127.151580


In [17]:
# 지도 시각화
import folium
from folium import Marker
from folium.plugins import MiniMap
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings('ignore')
from haversine import haversine, Unit

In [18]:
import seaborn as sns
from scipy.spatial import distance_matrix
from gurobipy import *
from scipy.spatial import ConvexHull
from shapely.geometry import Polygon, Point
from numpy import random
import time
from shapely import *
import requests
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl
import platform
from matplotlib import font_manager, rc
from shapely.geometry.point import Point
import re

In [19]:
# 파이썬 시각화 패키지 불러오기
#import matplotlib.pyplot as plt
%matplotlib inline

# 사용자 운영체제 확인
import platform
platform.system()

# 운영체제별 한글 폰트 설정
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정

# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [20]:
# FutureWarning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
# 서울시 중심부의 위도, 경도
center = [37.541, 126.986]

# 맵이 center 에 위치하고, zoom 레벨은 11로 시작하는 맵 m을 만듬
m3 = folium.Map(location=center, zoom_start=10)

In [22]:
# 서울특별시 송파구 데이터만 추출
es_address_sp = es_address_df[es_address_df['시군구명'].str.contains('송파구')]
es_address_sp.head()

,시군구명,주소,상세위치,위도,경도
126,송파구,가락동 58-3,경찰병원역 1번 출구 뒤,37.489567,127.120682
127,송파구,신천동 20-8,몽촌토성역 4번 출구 앞,37.517821,127.112477
128,송파구,송파동 1-1,석촌역 4번 출구 뒤,37.504790,127.113690
129,송파구,가락동 160-5,오금역 4번 출구 앞,37.496720,127.117440
130,송파구,잠실동 270,"잠실새내역 3, 4번 출구 사이",37.512281,127.085599


In [23]:
es_address_sp_lat = [x for x in es_address_sp['위도']]
es_address_sp_lon = [x for x in es_address_sp['경도']]

# m3은 지도에 전동 킥보드 거치대 유무(X)인 송파구
m3 = folium.Map(location=center, zoom_start=10)

for lat, lon in zip(es_address_sp_lat, es_address_sp_lon):
  folium.Marker(location = [lat,lon],
           icon=folium.Icon(color='red',icon='star')
          ).add_to(m3)
m3

In [24]:
# 지하철역(1호선~8호선) 위/경도
subway = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv', encoding = 'euc-kr')
subway = subway.drop(['작성일자'], axis=1)    # 열 삭제
subway.info()
subway.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            276 non-null    int64  
 1   호선            276 non-null    int64  
 2   고유역번호(외부역코드)  276 non-null    int64  
 3   역명            276 non-null    object 
 4   위도            276 non-null    float64
 5   경도            276 non-null    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 13.1+ KB


,연번,호선,고유역번호(외부역코드),역명,위도,경도
0,1,1,150,서울,37.553150,126.972533
1,2,1,151,시청,37.563590,126.975407
2,3,1,152,종각,37.570203,126.983116
3,4,1,153,종로3가,37.570429,126.992095
4,5,1,154,종로5가,37.570971,127.001900


In [25]:
# 서울시 전동킥보드 견인 현황
es_tow = pd.read_csv('/content/drive/MyDrive/캡스톤디자인/data/서울시 전동킥보드 견인 현황.csv', encoding = 'euc-kr')
es_tow.head()

,번호,신고일시,구정보,주소,유형,조치일
0,9999,2021-10-15,송파구,서울특별시 송파구 오금동 89-7,"점자블록, 엘리베이터 입구",2021-10-15
1,9998,2021-10-15,송파구,서울특별시 송파구 장지동 850,"점자블록, 엘리베이터 입구",2021-10-15
2,9997,2021-10-15,동작구,서울특별시 동작구 사당동 583-11,"점자블록, 엘리베이터 입구",2021-10-15
3,9996,2021-10-15,관악구,서울특별시 관악구 봉천동 979-1,지하철역 진출입로 및 주변,2021-10-15
4,9995,2021-10-15,관악구,서울특별시 관악구 봉천동 1693-39,"횡단보도, 산책로 등",2021-10-15


In [ ]:
import logging
logging.getLogger("urllib3").setLevel(logging.ERROR)

In [ ]:
es_tow_address = es_tow['주소']

# 위도, 경도 반환하는 함수
def geocoding(es_tow_address):
    try:
        geo = geo_local.geocode(es_tow_address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

#####주소를 위,경도 값으로 변환하기 #####
##### 오래걸림 ######
latitude_es_tow = []
longitude_es_tow = []

for i in es_tow_address:
    latitude_es_tow.append(geocoding(i)[0])
    longitude_es_tow.append(geocoding(i)[1])

#####Dataframe만들기######
es_tow_address_df = pd.DataFrame({'구정보': es_tow['구정보'],'주소': es_tow['주소'], '유형': es_tow['상세위치'], '위도':latitude_es_tow, '경도':longitude_es_tow})

In [ ]:
import geopandas as gpd
#from shapely.geometry import Polygon, LineString, Poin

In [ ]:
seoul_area = gpd.GeoDataFrame.from_file('/content/drive/MyDrive/캡스톤디자인/data/LARD_ADM_SECT_SGG_11.shp', encoding='cp949')
print(seoul_area.crs)
ax = seoul_area.plot(column="SGG_NM", figsize=(8,8), alpha=0.8)
ax.set_title("Seoul", fontsize=20)
ax.set_axis_off()

ERROR:fiona._env:Unable to open /content/drive/MyDrive/캡스톤디자인/data/LARD_ADM_SECT_SGG_11.shx or /content/drive/MyDrive/캡스톤디자인/data/LARD_ADM_SECT_SGG_11.SHX. Set SHAPE_RESTORE_SHX config option to YES to restore or create it.


DriverError: ignored